# Ejercicio con kNNs

Descargamos un dataset con información de Diabetes para nuestro ejercicio

In [33]:
!curl -o data/diabetes.csv https://raw.githubusercontent.com/plotly/datasets/master/diabetes.csv

% Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 23873  100 23873    0     0  26853      0 --:--:-- --:--:-- --:--:-- 26823
